<a href="https://colab.research.google.com/github/oiRudy/desafio_dio_sa_1_ETL/blob/main/SantanderDioETL2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Dio Santander (ETL com Python)

Esse projeto é uma adaptação do projeto do SantanderDevWeek2023, onde fui desafiado a modificar o que foi criado. As alterações foram feitas foi a troca do banco de dados, e substituindo a API rest criado tornando o codigo puramente em python.

**Contexto Antigo:** No ambiente do Santander, como cientista de dados, a missão é cativar os clientes com abordagens mais individuais. Através da IA Generativa, o foco é criar mensagens de marketing personalizadas, amplificando a conexão com cada cliente. O desafio envolve a manipulação de uma planilha CSV contendo IDs de clientes. O procedimento inclui acessar a API da Santander Dev Week 2023, através do endpoint GET , para coletar os detalhes de cada cliente. Com esses dados em mãos, a API do ChatGPT (OpenAI) é aproveitada para gerar mensagens que realçam a importância dos investimentos, adaptadas a cada cliente. A etapa final consiste em atualizar as notícias de cada cliente, utilizando o endpoint PUT, completando assim o ciclo personalizado de engajamento.

**Novo Contexto:** Este projeto abrange a análise de dados financeiros ao extrair informações históricas das ações do Banco Santander e, em seguida, transformar esses dados por meio de cálculos como a média móvel de 50 dias. Além disso, a API do GPT-3 é utilizada para gerar insights financeiros a partir dos dados transformados. A convergência entre análise quantitativa e linguagem natural permite uma compreensão abrangente dos padrões de preços, contribuindo para uma abordagem informada e inovadora na interpretação de dados financeiros.

## Entrada

In [19]:
# Entrada do usuário
symbol = "SANB4.SA" #Digite o símbolo da ação - Procurar na tabela de referenci do yahoo finance
action_name = "Satander" #Digite o nome da ação
start_date = "2022-01-01" #Digite a data de início no formato 'YYYY-MM-DD'
end_date = "2022-12-31"  #Digite a data de término no formato 'YYYY-MM-DD'
analysis_range = 50 #Digite o intervalo para a análise em dias

## **E**xtract

A primeira etapa deste projeto envolve a extração de dados históricos de preços das ações do Banco Santander. Isso é realizado usando a biblioteca yfinance do Python, que facilita o acesso aos dados do Yahoo Finance. Os dados extraídos incluem informações como preços de abertura, preços máximos, preços mínimos, preços de fechamento, preços ajustados e volume de negociação ao longo de um período especificado.

In [20]:
import pandas as pd
import yfinance as yf

# Etapa de Extração
data = yf.download(symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


## **T**ransform

Nesta etapa, os dados extraídos são processados e transformados para análises adicionais. Uma análise específica realizada é o cálculo da média móvel de 50 dias dos preços de fechamento das ações do Santander. A média móvel é uma medida que ajuda a suavizar flutuações de curto prazo e identificar tendências de médio prazo. Além disso, a etapa de transformação envolve a integração da API do GPT-3 para gerar insights financeiros com base nos dados transformados.

In [21]:
# Etapa de Transformação
# Calculando a média móvel do intervalo escolhido
data[f'SMA_{analysis_range}'] = data['Close'].rolling(window=analysis_range).mean()

In [ ]:
!pip install openai

In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto CHAVE por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'CHAVE'

In [ ]:
import openai

openai.api_key = openai_api_key

# Consulta ao GPT-3 para gerar um insight
insight_query = f"A média móvel de {analysis_range} dias dos preços da ação {action_name} está em {data[f'SMA_{analysis_range}'][-1]:.2f}. O que isso pode indicar sobre a tendência da ação?"
gpt_response = openai.Completion.create(
  engine="davinci",
  prompt=insight_query,
  max_tokens=50
)
gpt_insight = gpt_response.choices[0].text.strip()

## **L**oad

Após a transformação dos dados e a geração de insights usando o GPT-3, os resultados são carregados e organizados em um formato conveniente. Um DataFrame é criado para armazenar os dados transformados, incluindo os preços de fechamento, a média móvel de 50 dias e os insights gerados pelo GPT-3. Esses dados são então exportados para um arquivo CSV, que é um formato amplamente utilizado para compartilhamento e análise de dados.

In [ ]:
# Etapa de Carregamento
# Aqui você pode salvar os dados e os insights como um arquivo .csv
# Criando um DataFrame com dados e insights
data_transformed = data[['Date', 'Close', f'SMA_{analysis_range}']]
data_transformed['GPT_Insight'] = gpt_insight

# Salvando os dados transformados em um arquivo .csv
file_name = f'dados_transformados_{action_name}.csv'
data_transformed.to_csv(file_name, index=False)
print(f"Dados transformados salvos em '{file_name}'")